In [2]:
import numpy as np
import pandas as pd
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras.layers import LSTM, Dropout
from keras.layers import TimeDistributed
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping

from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import sys
import heapq
import seaborn as sns
from pylab import rcParams

import json
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [3]:
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential, Input
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

from keras.optimizers import RMSprop, Adam
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

## Gerar dataset

### - Bíblia

In [4]:
SEQUENCE_LENGTH = 40

In [5]:
with open('biblia.json') as f:
    content = f.read()

In [6]:
j = json.loads(content[1:]) # carrega string em um json

In [12]:
all_chaps = []
for book in j:
    for chap in book['chapters']:
        all_chaps.append(' '.join(chap))
        print(chap)
        break

['No princípio criou Deus os céus e a terra.', 'A terra era sem forma e vazia; e havia trevas sobre a face do abismo, mas o Espírito de Deus pairava sobre a face das águas.', 'Disse Deus: haja luz. E houve luz.', 'Viu Deus que a luz era boa; e fez separação entre a luz e as trevas.', 'E Deus chamou à luz dia, e às trevas noite. E foi a tarde e a manhã, o dia primeiro.', 'E disse Deus: haja um firmamento no meio das águas, e haja separação entre águas e águas.', 'Fez, pois, Deus o firmamento, e separou as águas que estavam debaixo do firmamento das que estavam por cima do firmamento. E assim foi.', 'Chamou Deus ao firmamento céu. E foi a tarde e a manhã, o dia segundo.', 'E disse Deus: Ajuntem-se num só lugar as águas que estão debaixo do céu, e apareça o elemento seco. E assim foi.', 'Chamou Deus ao elemento seco terra, e ao ajuntamento das águas mares. E viu Deus que isso era bom.', 'E disse Deus: Produza a terra relva, ervas que dêem semente, e árvores frutíferas que, segundo as suas

In [14]:
wordset = set()
for book in j:
    for chap in book['chapters']:
        for vers in chap:
            wordset.update(vers.split())

In [16]:
len(wordset)

53049

In [18]:
'profetizes' in wordset

True

In [8]:
X_data = []
y_data = []
for i in range(500_000):
    c = np.random.randint(0, len(all_chaps))
    curr_chap = all_chaps[c]
    a = np.random.randint(0, len(curr_chap) - 41)
    
    X_data.append(curr_chap[a : a + 40])
    y_data.append(curr_chap[a + 40])

In [9]:
for dado, alvo in zip(X_data[:10], y_data[:10]):
    print(dado, '-', alvo)

ue há em Cristo Jesus com glória eterna. -  
as, vivamos no presente mundo sóbria, e  - j
. Partindo dali, fomos navegando a sotav - e
vieram a Ezequias com as vestes rasgadas - ,
rolados, da idade de vinte anos e acima, -  
 moças, como a irmãs, com toda a pureza. -  
o vinho de furor, e faze que dele bebam  - t
, e favas, e lentilhas, e milho miúdo, e -  
uste será pior do que o primeiro. Disse- - l
 abundância de suas delícias. Ouvi outra -  


In [9]:
tokenizer = Tokenizer(
    num_words=500, # número máximo - valor grande arbitrário - se tirar não funciona - não sei pq
    char_level=True,
    filters=None,
    lower=False,
    oov_token=chr(1) # símbolo com quadradinho - funciona no js
)

In [10]:
%%time

tokenizer.fit_on_texts(X_data)
tokenizer.fit_on_texts(y_data)
# word_index = tokenizer.word_index
# index_word = tokenizer.index_word

print('Found %s unique tokens.' % len(tokenizer.word_index))

Found 89 unique tokens.
CPU times: user 3.65 s, sys: 0 ns, total: 3.65 s
Wall time: 3.65 s


In [11]:
X_data_bibl40 = X_data
y_data_bibl40 = y_data
tok_bibl = tokenizer

## Treinar modelo

In [12]:
X = np.array(tokenizer.texts_to_sequences(X_data), dtype=np.int32)
y = np.array(list(map(tokenizer.word_index.get, y_data)))

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [14]:
num_chars = len(tokenizer.index_word) + 1

In [26]:
emb_dim = 256
seq_units = 128 # faz muita diferença diminuir
drop = .1

model = Sequential()
model.add(Embedding(num_chars, emb_dim, input_length=X_train.shape[1]))
# model.add(SpatialDropout1D(drop)) # maybe should remove because of rare words...
model.add(Bidirectional(GRU(seq_units, return_sequences=True, dropout=0.1,recurrent_dropout=0.1)))
model.add(GlobalMaxPool1D())

model.add(Dense(num_chars, activation='softmax'))

In [27]:
model.compile(
    optimizer=RMSprop(lr=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 40, 256)           23040     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 40, 256)           295680    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 90)                23130     
Total params: 341,850
Trainable params: 341,850
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
%%time
n = len(X_train) // 1
# change val split to .1 or .05?

epochs = 400
batch_size = 1024
history = model.fit(X_train[:n], y_train[:n], epochs=epochs, batch_size=batch_size,
                    validation_split=0.2,
#                     sample_weight = sample_weight[:n],
                    callbacks=[EarlyStopping(monitor='val_loss',patience=1, min_delta=1e-7)],
                   )

Train on 320000 samples, validate on 80000 samples
Epoch 1/400
320000/320000 [==============================] - 21s 65us/step - loss: 2.4919 - sparse_categorical_accuracy: 0.3046 - val_loss: 2.1925 - val_sparse_categorical_accuracy: 0.3555
Epoch 2/400
320000/320000 [==============================] - 20s 62us/step - loss: 2.1084 - sparse_categorical_accuracy: 0.3794 - val_loss: 2.0361 - val_sparse_categorical_accuracy: 0.3952
Epoch 3/400
320000/320000 [==============================] - 20s 62us/step - loss: 1.9827 - sparse_categorical_accuracy: 0.4080 - val_loss: 1.9301 - val_sparse_categorical_accuracy: 0.4182
Epoch 4/400
320000/320000 [==============================] - 20s 62us/step - loss: 1.8986 - sparse_categorical_accuracy: 0.4308 - val_loss: 1.8767 - val_sparse_categorical_accuracy: 0.4287
Epoch 5/400
320000/320000 [==============================] - 20s 62us/step - loss: 1.8372 - sparse_categorical_accuracy: 0.4495 - val_loss: 1.8020 - val_sparse_categorical_accuracy: 0.4593
Epoc

### Salvar modelo

In [36]:
model.save('biblia_40-keras_model.h5')
# pickle.dump(history, open("biblia-history.p", "wb"))
with open('biblia_40-index_word.json', 'w') as f:
    json.dump(tokenizer.index_word, f, ensure_ascii=False)
with open('biblia_40-word_index.json', 'w') as f:
    json.dump(tokenizer.word_index, f, ensure_ascii=False)

In [30]:
# with open('biblia-word_index.json', 'r') as f:
#     x = json.load(f)
# x

In [31]:
# model = load_model('keras_model.h5')
# history = pickle.load(open("history.p", "rb"))

### Testar Modelo

In [32]:
curr = X_test[0].copy().reshape((1, -1))
original = ''.join(map(tokenizer.index_word.get, curr[0]))

In [33]:
next_seq = []
while True:
    _next = model.predict_classes(curr)[0]
    next_char = tokenizer.index_word[_next]
    if next_char == ' ':
        break
    next_seq.append(next_char)
    curr[0, 0 : -1] = curr[0, 1 :]
    curr[0, -1] = _next


In [34]:
print(original, ''.join(next_seq))

em viagem, deixar de celebrar a páscoa,  e


In [35]:
for i in range(30):
    curr = X_test[i].copy().reshape((1, -1))
    original = ''.join(map(tokenizer.index_word.get, curr[0]))
    blank_count = 0
    next_seq = []
    while True:
        _next = model.predict_classes(curr)[0]
        next_char = tokenizer.index_word[_next]
        if next_char == ' ':
            blank_count += 1
        if blank_count == 6:
            break
        next_seq.append(next_char)
        curr[0, 0 : -1] = curr[0, 1 :]
        curr[0, -1] = _next
        
    print(original, '--', ''.join(next_seq))

em viagem, deixar de celebrar a páscoa,  -- e a sua casa de servo
 que peregrinar o estrangeiro, ali lhe d -- e terra de todas as suas
lhe chamarás mais Sarai, porem Sara será --  para se profeta e a
vida, que produz doze frutos, dando seu  -- pais e todos os seus profetisses
te será proveitoso. Tem o Todo-Poderoso  -- a terra de todas as suas
vir as palavras do Senhor. Andarão erran -- te a sua casa de Deus,
 de seus filhos com ele. E disse Moisés  -- se fazer com eles servir a
porém te darei a tua vida por despojo, e --  a terra de todos os
andamentos e os meus estatutos. Mas da m -- inha casa de Deus de Israel,
 nele habitam. Que aproveita a imagem es -- ta casa de Deus, e o
companhia dos sacerdotes que matam no ca -- minho de terra de transgressão a
se apanham com a rede maligna, e como os --  seus profetisse a sua casa
a que o Senhor teu Deus te dá, não apren -- tas a sua casa de Deus
tuas tendas; então levantarás o teu rost -- o e todos os seus profetisses
us de Israel: Eis que t

In [ ]:
companhia dos sacerdotes que matam no ca & minho de terra de transgressão a \\
se apanham com a rede maligna, e como os &  seus profetisse a sua casa \\
a que o Senhor teu Deus te dá, não apren & tas a sua casa de Deus \\
tuas tendas; então levantarás o teu rost & o e todos os seus profetisses \\
us de Israel: Eis que trarei sobre este  & pergunto e a terra de Cristo, \\
odo o seu trabalho é dom de Deus. Eu sei &  o Senhor se alguma a \\
e me instruíam inclinei o meu ouvido! Qu & ando se fazer a sua casa \\
njo: Não estendas a mão sobre o mancebo, &  e todos os seus casas \\
sando bem cada oportunidade. A vossa pal & avra do Senhor Deus de Israel \\

In [ ]:
ntar summariamente um galante episodio d & a lha filha virgilia a talva \\
 não importa a edade do adulado; a mulh  & a trimagilia palama, tomantavamas a parta \\
que ha, quando se não resolve um enigma, &  em aguita filha pata pala \\
der nada, a principio, depois entendendo &  que tinha filha ficação, que \\
al é a materia do capitulo anterior. pod & a tem filha pala com filha \\
enha cá jantar amanhã, disse-me o dutra  & uma nala. trava uma cala a \\
ão sei se contava commigo na sala; mas e & ma filha jatalha, a viva á \\
ças de roupa, a batel-as, a ensaboal-as, &  com a malha palação,— filha \\
iro chinez, bojudo, destro, escanhoando  & com a cara, a fica, e \\
versos, e com elles ia tambem passando a &  milha para para pala para \\
lhar interrogativo ou suspeitoso. pergun & tava a lama a lama a \\